# Iteration

Iteration is generally goverened by two types of objects. _Iterables_ and _Iterators_. _Iterables_ is a type of object that follows a certain protocol which allows it to be iterated over, for example using a for loop to traverse its contents. An _iterator_ is a type of object that manages the process of producing certain values one at a time.

In [1]:
for x in [1, 2, 3, 'a', 'b', 'c']:
    print(x, end=' ') # lists are iterable

1 2 3 a b c 

We can look at the underlying mechanism by which loops might perform iteration. We can create an iterator from our iterable, and ask it to produce sequential values by calling `next`

In [2]:
some_numbers = [1, 2, 3]
it = iter(some_numbers)

In [3]:
next(it)

1

In [4]:
next(it)

2

In [5]:
next(it)

3

In [6]:
next(it)

StopIteration: 

Once an iterator fails to yield further values to us, it raises `StopIteration` and that's how we know it has completed.

### Building an Iterable

Now suppose we want to construct an object that is iterable, or implements the _iterable protocol_. There are fundamentally two required methods:

- `__next__(self)` - returns next element from an iterator, or raises `StopIteration`

- `__iter__(self)` - constructs and returns an iterator, to iterate over itself

An **iterable** must simply implement the latter that returns an iterator.
An **iterator** implements _both_ methods. Even though the iterator itself is already assumed to implement next, if we try to use something like a for loop, python attempts to call `iter()` on that object.

(Alternatively implementing `__len__` and `__getitem__` also makes an object iterable)

We can observe how lists generally follow this behavior:

In [13]:
x = some_numbers.__iter__()
x.__next__()

1

A good example given in the notes is building something similar to the builtin `range()` method in Python. Here is a simplified version.

In [1]:
class FauxRangeIter:
    def __init__(self, faux_range):
        self._faux_range = faux_range
        self._cur = faux_range.start

    def __iter__(self):
        return self

    def __next__(self):
        if self._cur >= self._faux_range.stop:
            raise StopIteration
        else:
            temp = self._cur
            self._cur += self._faux_range.step
            return temp

class FauxRange:
    def __init__(self, start, stop = None, step = None):
        if stop is None:
            stop = start
            start = 0
        
        if step is None:
            step = 1

        self.start = start
        self.stop = stop
        self.step = step
    
    def __iter__(self):
        return FauxRangeIter(self)

For the most part, this will behave similarly to an actual `range`, although it isn't a generator. When a for loop iterates over a `FauxRange`, it gets an iterator by calling `__iter__`, then sequentially calls next until it hits a StopIteration

In [2]:
for x in FauxRange(3, 10, 2):
    print(x, end=' ')

3 5 7 9 

It's also worth noting that pretty much all of the methods in the two classes above run in $O(1)$ time, because if we lazily produce values as we need them on calls of `__next__`, we can be considerably more efficient than were we to figure out all of the values first and store them somewhere.